In [1]:
from helpers import load_all_datasets, compose, str_rep, str_rep_no_steps, CBRDatabase
import pandas as pd

DATABASE_NAME = "cbr_database"
OUTPUT_FILE = "cbr_augmentation_2_input.csv"

In [2]:
additional_cbr_1, _, _, _, cbr_database, _, _ = load_all_datasets()
cbr_database_id = cbr_database.set_index("id")

In [3]:
cbr_database_reps = str_rep(cbr_database)

In [4]:
# meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_database["id"].values)], {"dataset":"cbr_database"})

# vdb = CBRDatabase(cbr_database_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)
vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [5]:


# [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query="xd", k=4)]

In [6]:
# generowanie inputów - 1500 tokenów 
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in additional_cbr_1.iterrows():
    while True:
        # recipe_ixs = [random.randint(0, len(cbr_database)-1) for _ in range(7)]
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=7)]

        gen_input = compose(cbr_database_id.loc[recipe_ixs], v_recipe)
        
        if len(tokenizer.encode(gen_input)) <= 1550: 
            br = True
        else:
            for ii in list(range(6, -1, -1)):
                gen_input = compose(cbr_database_id.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 1550: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            for ii in list(range(5, -1, -1)):
                gen_input = compose(cbr_database_id.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 1550: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_database_id.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 - 1500
2 - 1337
1 - 1499
2 - 1210
1 - 1318
2 - 1109
1 - 1393
2 - 1161
1 - 1237
2 - 993
1 - 1405
2 - 1207
1 - 1415
2 - 1159
1 - 1373
2 - 1161
1 - 1543
2 - 1345
1 - 1361
2 - 1117
1 - 1283
2 - 1080
1 - 1399
2 - 1200
1 - 1414
2 - 1160
1 - 1432
2 - 1222
1 - 1305
2 - 1090
1 - 1340
2 - 1131
1 - 1500
2 - 1336
1 - 1311
2 - 1111
1 - 1353
2 - 1072
1 - 1456
2 - 1293
1 - 1415
2 - 1242
1 - 1502
2 - 1237
1 - 1371
2 - 1171
1 - 1416
2 - 1167
1 - 1434
2 - 1245
1 - 1313
2 - 1169
1 - 1433
2 - 1178
1 - 1340
2 - 1106
1 - 1510
2 - 1270
1 - 1382
2 - 1215
1 - 1386
2 - 1132
1 - 1412
2 - 1229
1 - 1463
2 - 1274
1 - 1328
2 - 1171


In [7]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}", index=False)

In [8]:
cbr_database_id.loc[[227451, 227451]]

,name,steps,ingredients
id,,,
227451,russian berry custard kissel,Place the berries and 4 cups of the water in a...,"fresh cranberries, water, sugar, potato starch"
227451,russian berry custard kissel,Place the berries and 4 cups of the water in a...,"fresh cranberries, water, sugar, potato starch"
